# Modèle "ouvert"

L'objet de ce notebook est de démontrer la faisabilité de prédire les listes d'ingrédients depuis des fiches techniques

## Préambule technique

In [1]:
# setting up sys.path for relative imports
from pathlib import Path
import sys
project_root = str(Path(sys.path[0]).parents[1].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# imports and customization of diplay
# import os
# from functools import partial
import numpy as np
import pandas as pd
pd.options.display.min_rows = 6
pd.options.display.width=108
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.pipeline import Pipeline
# from matplotlib import pyplot as plt

from src.pimapi import Requester
from src.pimest import PIMIngredientExtractor
# from src.pimest import ContentGetter
# from src.pimest import PathGetter
# from src.pimest import PDFContentParser
# from src.pimest import BlockSplitter
# from src.pimest import SimilaritySelector
# from src.pimest import custom_accuracy

In [3]:
# monkeypatch _repr_latex_ for better inclusion of dataframes output in report
def _repr_latex_(self, size='scriptsize',):
    return(f"\\resizebox{{\\linewidth}}{{!}}{{\\begin{{{size}}}\\centering{{{self.to_latex()}}}\\end{{{size}}}}}")
pd.DataFrame._repr_latex_ = _repr_latex_

## Extraction des données

On extrait les données depuis le PIM :

In [4]:
requester = Requester('prd')
requester.fetch_all_from_PIM()
requester.result

Done


[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [5]:
df = requester.result_to_dataframe(record_path='entries', index='uid')

## Constitution du périmètre

On conserve les produits qui : 
- sont de type Epicerie ou Boisson non alcoolisée
- portent une liste d'ingrédients
- sont en qualité :
    - soit ont terminé le processus de migration, soit ont été créés après la reprise initiale
    - et ont le statut "Validé"

In [6]:
# filter by product type
type_mask = df['properties.pprodtop:typeOfProduct'].isin(['grocery', 'nonAlcoholicDrink'])

# keep only those who have ingredients
ingredient_mask = pd.notna(df['properties.pprodc:ingredientsList'])

# filter out those who have not finished migration
df['begin_mig'] = df['facets'].apply(lambda x: 'beginningMigration' in x)
df['end_mig'] = df['facets'].apply(lambda x: 'endMigration' in x)
migration_mask = df.loc[:, 'end_mig'] | ~df.loc[:, 'begin_mig']

# filter out those who are not validated
status_mask = (df.loc[:, 'state'] == 'product.validate')

scope_mask = type_mask & ingredient_mask & migration_mask & status_mask

scope_df = df.loc[scope_mask]
print(f'After filters, there are {len(scope_df)} records in the dataset,')
out_of_scope_df = df.loc[~df.index.isin(scope_df.index)]
print(f'and {len(out_of_scope_df)} records left out.')

After filters, there are 3407 records in the dataset,
and 9893 records left out.


## Entraînement : constitution du vocabulaire

On entraîne le modèle sur les listes d'ingrédients du périmètre. Cela revient à fitter le CountVectorizer sous-jacent. 

In [7]:
model = PIMIngredientExtractor('prd')
model.fit(scope_df['properties.pprodc:ingredientsList'])

On peut imprimer une partie du vocabulaire qui a été construit :

In [8]:
print(f'Vocabulary consists in {len(model._count_vect.vocabulary_)} words.\n')
print('Some words examples are :')

for i, word in enumerate(model._count_vect.vocabulary_.keys()):
    print('- ', word)
    if i > 6:
        break

Vocabulary consists in 2514 words.

Some words examples are :
-  morilles
-  kombu
-  déshydraté
-  100
-  eau
-  graines
-  de
-  moutarde


On peut également afficher les mots les plus fréquents dans le corpus de listes d'ingrédients d'entraînement. On constitue d'abord la matrice des textes transformés :

In [9]:
vectorized = model._count_vect.transform(scope_df['properties.pprodc:ingredientsList'])
vectorized.shape

(3407, 2514)

On a bien 3412 documents projetés sur 2509 mots. Si on extrait les plus fréquents, on obtient :

In [10]:
inverse_voc = {val: key for key, val in model._count_vect.vocabulary_.items()}
word_counts = np.asarray(vectorized.sum(axis=0)).squeeze()
print('Most frequent words in vocabulary are:')
for idx in word_counts.argsort()[::-1][:10]:
    print(f'{inverse_voc[idx].ljust(7)}: {word_counts[idx]:5} occurences')


Most frequent words in vocabulary are:
de     : 11544 occurences
sucre  :  2069 occurences
sel    :  1647 occurences
eau    :  1266 occurences
acide  :  1245 occurences
huile  :  1241 occurences
lait   :  1228 occurences
poudre :  1099 occurences
en     :   972 occurences
arôme  :   940 occurences


## Prédictions

Le wrapper `PIMIngredientExtractor` permet de simplement récupérer les informations du PIM et les pièces jointes associées, et de faire tourner le modèle pour extraire le bloc le plus similaire aux listes d'ingrédients.

In [20]:
exec_count = 5
uids = list(out_of_scope_df.sample(exec_count, ).index)
i = 0

for uid in uids:
        try:
            model.compare_uid_data(uid)
            print('\n=========================================================='
                   '\n==========================================================\n')
            i += 1
        except:
            pass
        if i > exec_count:
            break

Fetching data from PIM for uid 78dd3d32-5c23-495e-a4f9-71f6b3c692d7...
Done
----------------------------------------------------------
Ingredient list from PIM is :

Sens gourmet Sarl - 15/17 rue du travy  ZI sénia 715  94657 Thiais T 01 49 79 98 29  F 01 48 85 36 32  info@sensgourmet.com Code 58050000 (500 gr) Code EAN 8414933570004 RSIPAC N° 31-04482/CAT-31.01506B DESCRIPTION : Ingrédient d'origine naturel. Agent gélifiant pour des gelées complétement transparentes à base d'eau. Origine : Europe APPLICATIONS : La gélatine est un ingrédient d'origine naturelle qui a une grande capacité d'absorption des molécules d'eau. Il s'agit d'une gélatine très élastique avec une bonne absorption de l'eau. Une température de gélification : 60ºC. Dosage : 50 g/kg COMPOSITION : Maltodextrine, agent épaississant: carrageenan (E407), dextrose, chlorure de potassium (E508), acidifiant: trisodium citrate (E331iii), agent épaississant: gomme de caroube (E410), saccharose.

-------------------------------